In [1]:
from helper import get_training_data, train_lasso_regression, plot_result_correlation, get_best_lasso_model, train_xgboost,  get_best_pca_lasso_model, train_mean_model
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn import metrics
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
from helper import prepare_socios

%matplotlib inline
%load_ext autoreload
%autoreload 2
pd.options.mode.chained_assignment = None
import warnings
warnings.filterwarnings('ignore')

## Best models

#### Unemployment rate

In [2]:
results = pd.DataFrame(columns = ['improvement_lasso', 'improvement_lasso_boosted', 'improvement_pca_lasso', 'improvement_pca_lasso_boosted', 'improvement_mean_model'])
results_mse = pd.DataFrame(columns = ['mse_naive','mse_lasso', 'mse_lasso_boosted', 'mse_pca_lasso', 'mse_pca_lasso_boosted', 'mse_mean_model'])
results_r2 = pd.DataFrame(columns = ['r2_naive', 'r2_lasso', 'r2_lasso_boosted', 'r2_pca_lasso', 'r2_pca_lasso_boosted', 'r2_mean_model'])
target = 'unemployment_rate'
dens_type = 'score'
radius = 1000

for city in ['marseille', 'lyon', 'paris']:
    country = 'FR'
    agg = get_training_data(city, country, radius, dens_type,  2015)
    
    # Lasso:
    predicts = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='predicts')
    predicts_master = predicts.copy()
    predicts_master = predicts_master.rename(columns = {'y_pred': 'y_pred_lasso'})

    # Lasso Boosted
    cols = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='used_columns')
    predicts = train_xgboost(agg, target, cols, 'predicts')
    predicts_master.loc[:, 'y_pred_lasso_boosted'] = predicts.y_pred

    # PCA Lasso
    predicts = get_best_pca_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius,output = 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso'] = predicts.y_pred

    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius, output = 'components')
    cols = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius, output = 'used_columns')
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(agg.iloc[:,-5:])
    predicts = train_xgboost(reduced_data, target, cols, 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso_boosted'] = predicts.y_pred

    #predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso', 'y_pred_lasso_boosted', 'y_pred_pca_lasso', 'y_pred_pca_lasso_boosted']].mean(axis = 1)
    predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted']].mean(axis = 1)


    targets = ['unemployment_rate', 'income_levels', 'foreign_nationals']
    scaler = get_training_data(city, country, radius, dens_type, 2015, 'scaler')
    scaler_new = RobustScaler()
    scaler_new.center_, scaler_new.scale_ = scaler.center_[targets.index(target)], scaler.scale_[targets.index(target)]
    predicts_master = pd.DataFrame(scaler_new.inverse_transform(predicts_master), columns =predicts_master.columns)


    naive_mse = metrics.mean_squared_error(predicts_master.y_test, predicts_master.naive) 
    mse_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso)
    mse_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    mse_pca_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    mse_pca_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    mse_mean_model = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_mean_model)
    
    naive_r2 = metrics.r2_score(predicts_master.y_test, predicts_master.naive) 
    r2_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso)
    r2_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    r2_pca_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    r2_pca_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    r2_mean_model = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_mean_model)

    improvement_lasso = 100 - (mse_lasso/naive_mse)*100
    improvement_lasso_boosted = 100 - (mse_lasso_boosted /naive_mse)*100
    improvement_pca_lasso = 100 - (mse_pca_lasso/naive_mse)*100
    improvement_pca_lasso_boosted = 100 - (mse_pca_lasso_boosted/naive_mse)*100
    improvement_mean_model = 100 - (mse_mean_model/naive_mse)*100
    results.loc[city,:]=[improvement_lasso, improvement_lasso_boosted, improvement_pca_lasso, improvement_pca_lasso_boosted, improvement_mean_model]
    results_mse.loc[city,:]=[naive_mse, mse_lasso, mse_lasso_boosted, mse_pca_lasso, mse_pca_lasso_boosted, mse_mean_model]
    results_r2.loc[city,:]=[naive_r2, r2_lasso, r2_lasso_boosted, r2_pca_lasso, r2_pca_lasso_boosted, r2_mean_model]
    
for city in ['hamburg', 'berlin', 'bremen']:
    country = 'DE'
    agg = get_training_data(city, country, radius, dens_type,  2015)
    
    # Lasso:
    predicts = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='predicts')
    predicts_master = predicts.copy()
    predicts_master = predicts_master.rename(columns = {'y_pred': 'y_pred_lasso'})

    # Lasso Boosted
    cols = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='used_columns')
    predicts = train_xgboost(agg, target, cols, 'predicts')
    predicts_master.loc[:, 'y_pred_lasso_boosted'] = predicts.y_pred

    # PCA Lasso
    predicts = get_best_pca_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius,output = 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso'] = predicts.y_pred

    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius, output = 'components')
    cols = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output = 'used_columns')
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(agg.iloc[:,-5:])
    predicts = train_xgboost(reduced_data, target, cols, 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso_boosted'] = predicts.y_pred

    #predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso', 'y_pred_lasso_boosted', 'y_pred_pca_lasso', 'y_pred_pca_lasso_boosted']].mean(axis = 1)
    predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted']].mean(axis = 1)

    targets = ['unemployment_rate', 'income_levels', 'foreign_nationals']
    scaler = get_training_data(city, country, radius, dens_type, 2015, 'scaler')
    scaler_new = RobustScaler()
    scaler_new.center_, scaler_new.scale_ = scaler.center_[targets.index(target)], scaler.scale_[targets.index(target)]
    predicts_master = pd.DataFrame(scaler_new.inverse_transform(predicts_master), columns =predicts_master.columns)


    naive_mse = metrics.mean_squared_error(predicts_master.y_test, predicts_master.naive) 
    mse_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso)
    mse_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    mse_pca_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    mse_pca_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    mse_mean_model = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_mean_model)
    
    naive_r2 = metrics.r2_score(predicts_master.y_test, predicts_master.naive) 
    r2_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso)
    r2_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    r2_pca_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    r2_pca_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    r2_mean_model = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_mean_model)

    improvement_lasso = 100 - (mse_lasso/naive_mse)*100
    improvement_lasso_boosted = 100 - (mse_lasso_boosted /naive_mse)*100
    improvement_pca_lasso = 100 - (mse_pca_lasso/naive_mse)*100
    improvement_pca_lasso_boosted = 100 - (mse_pca_lasso_boosted/naive_mse)*100
    improvement_mean_model = 100 - (mse_mean_model/naive_mse)*100
    results.loc[city,:]=[improvement_lasso, improvement_lasso_boosted, improvement_pca_lasso, improvement_pca_lasso_boosted, improvement_mean_model]
    results_mse.loc[city,:]=[naive_mse, mse_lasso, mse_lasso_boosted, mse_pca_lasso, mse_pca_lasso_boosted, mse_mean_model]
    results_r2.loc[city,:]=[naive_r2, r2_lasso, r2_lasso_boosted, r2_pca_lasso, r2_pca_lasso_boosted, r2_mean_model]

results.loc['mean'] = results.mean()
results

shape of training data (344, 238)
0.1
0.1
shape before pca: (344, 231)
number of pca components: 50
shape after pca: (344, 50)
0.1
shape before pca: (344, 231)
number of pca components: 50
shape after pca: (344, 50)
0.1
shape before pca: (344, 231)
number of pca components: 50
shape after pca: (344, 50)
0.1
shape of training data (344, 238)
shape of training data (176, 240)
0.2
0.2
shape before pca: (176, 233)
number of pca components: 49
shape after pca: (176, 49)
0.1
shape before pca: (176, 233)
number of pca components: 49
shape after pca: (176, 49)
0.1
shape before pca: (176, 233)
number of pca components: 49
shape after pca: (176, 49)
0.1
shape of training data (176, 240)
shape of training data (861, 241)
0.01
0.01
shape before pca: (861, 234)
number of pca components: 118
shape after pca: (861, 118)
0.01
shape before pca: (861, 234)
number of pca components: 118
shape after pca: (861, 118)
0.01
shape before pca: (861, 234)
number of pca components: 118
shape after pca: (861, 118)

,improvement_lasso,improvement_lasso_boosted,improvement_pca_lasso,improvement_pca_lasso_boosted,improvement_mean_model
marseille,46.96895,32.458085,45.459434,45.393337,44.88
lyon,22.507703,32.303641,28.270945,22.693494,33.243819
paris,34.602947,31.307113,34.573061,30.650135,34.821246
hamburg,41.466271,49.396633,36.258663,10.08553,40.608638
berlin,27.25215,33.055558,38.968953,25.910502,34.305281
bremen,38.22746,49.44615,48.393576,46.093483,53.835441
mean,35.170914,37.99453,38.654105,30.137747,40.282404


In [3]:
results_r2

,r2_naive,r2_lasso,r2_lasso_boosted,r2_pca_lasso,r2_pca_lasso_boosted,r2_mean_model
marseille,-0.001364,0.468966,0.32366,0.453851,0.453189,0.448048
lyon,-0.001873,0.223625,0.321768,0.281366,0.225487,0.331188
paris,-0.014116,0.336798,0.303374,0.336495,0.296712,0.339012
hamburg,-0.124421,0.341835,0.431005,0.283279,-0.011017,0.332191
berlin,-0.00026,0.272333,0.330382,0.389531,0.258913,0.342882
bremen,-0.043206,0.355585,0.472619,0.461639,0.437644,0.518409


In [4]:
results_mse

,mse_naive,mse_lasso,mse_lasso_boosted,mse_pca_lasso,mse_pca_lasso_boosted,mse_mean_model
marseille,0.010888,0.005774,0.007354,0.005938,0.005945,0.006001
lyon,0.002615,0.002026,0.00177,0.001876,0.002021,0.001746
paris,0.001694,0.001108,0.001164,0.001108,0.001175,0.001104
hamburg,0.000365,0.000214,0.000185,0.000233,0.000328,0.000217
berlin,0.000859,0.000625,0.000575,0.000524,0.000636,0.000564
bremen,0.001918,0.001185,0.00097,0.00099,0.001034,0.000886


In [5]:
results_rmse = results_mse**(0.5)
results_rmse = results_rmse.rename(columns = {'mse_naive': 'rmse_naive','mse_lasso':'rmse_lasso', 'mse_lasso_boosted':'rmse_lasso_boosted', 'mse_pca_lasso':'rmse_pca_lasso', 'mse_pca_lasso_boosted':'rmse_pca_lasso_boosted', 'mse_mean_model':'rmse_mean_model'})
results_rmse

,rmse_naive,rmse_lasso,rmse_lasso_boosted,rmse_pca_lasso,rmse_pca_lasso_boosted,rmse_mean_model
marseille,0.104344,0.075986,0.085754,0.077059,0.077106,0.077468
lyon,0.051135,0.045014,0.042073,0.043308,0.04496,0.04178
paris,0.041161,0.033286,0.034115,0.033294,0.034277,0.033231
hamburg,0.0191,0.014613,0.013587,0.015249,0.018112,0.01472
berlin,0.029304,0.024994,0.023976,0.022893,0.025224,0.023752
bremen,0.043799,0.034424,0.031142,0.031464,0.032158,0.029759


In [6]:
output = pd.concat([results, results_r2, results_mse, results_rmse], axis = 1, join = 'inner')
output = output[['mse_naive', 'rmse_naive', 'r2_naive',
               'mse_lasso', 'rmse_lasso', 'r2_lasso', 'improvement_lasso',
               'mse_lasso_boosted', 'rmse_lasso_boosted', 'r2_lasso_boosted', 'improvement_lasso_boosted',
               'mse_pca_lasso', 'rmse_pca_lasso', 'r2_pca_lasso', 'improvement_pca_lasso',
               'mse_pca_lasso_boosted', 'rmse_pca_lasso_boosted', 'r2_pca_lasso_boosted', 'improvement_pca_lasso_boosted',
               'mse_mean_model', 'rmse_mean_model', 'r2_mean_model', 'improvement_mean_model']]
output.to_excel(f'output/results/results_{target}_{dens_type}_{radius}.xlsx')
output

,mse_naive,rmse_naive,r2_naive,mse_lasso,rmse_lasso,r2_lasso,improvement_lasso,mse_lasso_boosted,rmse_lasso_boosted,r2_lasso_boosted,...,r2_pca_lasso,improvement_pca_lasso,mse_pca_lasso_boosted,rmse_pca_lasso_boosted,r2_pca_lasso_boosted,improvement_pca_lasso_boosted,mse_mean_model,rmse_mean_model,r2_mean_model,improvement_mean_model
marseille,0.010888,0.104344,-0.001364,0.005774,0.075986,0.468966,46.96895,0.007354,0.085754,0.32366,...,0.453851,45.459434,0.005945,0.077106,0.453189,45.393337,0.006001,0.077468,0.448048,44.88
lyon,0.002615,0.051135,-0.001873,0.002026,0.045014,0.223625,22.507703,0.00177,0.042073,0.321768,...,0.281366,28.270945,0.002021,0.04496,0.225487,22.693494,0.001746,0.04178,0.331188,33.243819
paris,0.001694,0.041161,-0.014116,0.001108,0.033286,0.336798,34.602947,0.001164,0.034115,0.303374,...,0.336495,34.573061,0.001175,0.034277,0.296712,30.650135,0.001104,0.033231,0.339012,34.821246
hamburg,0.000365,0.0191,-0.124421,0.000214,0.014613,0.341835,41.466271,0.000185,0.013587,0.431005,...,0.283279,36.258663,0.000328,0.018112,-0.011017,10.08553,0.000217,0.01472,0.332191,40.608638
berlin,0.000859,0.029304,-0.00026,0.000625,0.024994,0.272333,27.25215,0.000575,0.023976,0.330382,...,0.389531,38.968953,0.000636,0.025224,0.258913,25.910502,0.000564,0.023752,0.342882,34.305281
bremen,0.001918,0.043799,-0.043206,0.001185,0.034424,0.355585,38.22746,0.00097,0.031142,0.472619,...,0.461639,48.393576,0.001034,0.032158,0.437644,46.093483,0.000886,0.029759,0.518409,53.835441


#### Foreign Nationals

In [7]:
results = pd.DataFrame(columns = ['improvement_lasso', 'improvement_lasso_boosted', 'improvement_pca_lasso', 'improvement_pca_lasso_boosted', 'improvement_mean_model'])
results_mse = pd.DataFrame(columns = ['mse_naive','mse_lasso', 'mse_lasso_boosted', 'mse_pca_lasso', 'mse_pca_lasso_boosted', 'mse_mean_model'])
results_r2 = pd.DataFrame(columns = ['r2_naive', 'r2_lasso', 'r2_lasso_boosted', 'r2_pca_lasso', 'r2_pca_lasso_boosted', 'r2_mean_model'])
target = 'foreign_nationals'
dens_type = 'score'
radius = 1000

for city in ['marseille', 'lyon', 'paris']:
    country = 'FR'
    agg = get_training_data(city, country, radius, dens_type,  2015)
    
    # Lasso:
    predicts = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='predicts')
    predicts_master = predicts.copy()
    predicts_master = predicts_master.rename(columns = {'y_pred': 'y_pred_lasso'})

    # Lasso Boosted
    cols = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='used_columns')
    predicts = train_xgboost(agg, target, cols, 'predicts')
    predicts_master.loc[:, 'y_pred_lasso_boosted'] = predicts.y_pred

    # PCA Lasso
    predicts = get_best_pca_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius,output = 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso'] = predicts.y_pred

    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius, output = 'components')
    cols = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius, output = 'used_columns')
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(agg.iloc[:,-5:])
    predicts = train_xgboost(reduced_data, target, cols, 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso_boosted'] = predicts.y_pred

    #predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso', 'y_pred_lasso_boosted', 'y_pred_pca_lasso', 'y_pred_pca_lasso_boosted']].mean(axis = 1)
    predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted']].mean(axis = 1)


    targets = ['unemployment_rate', 'income_levels', 'foreign_nationals']
    scaler = get_training_data(city, country, radius, dens_type, 2015, 'scaler')
    scaler_new = RobustScaler()
    scaler_new.center_, scaler_new.scale_ = scaler.center_[targets.index(target)], scaler.scale_[targets.index(target)]
    predicts_master = pd.DataFrame(scaler_new.inverse_transform(predicts_master), columns =predicts_master.columns)


    naive_mse = metrics.mean_squared_error(predicts_master.y_test, predicts_master.naive) 
    mse_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso)
    mse_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    mse_pca_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    mse_pca_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    mse_mean_model = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_mean_model)
    
    naive_r2 = metrics.r2_score(predicts_master.y_test, predicts_master.naive) 
    r2_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso)
    r2_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    r2_pca_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    r2_pca_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    r2_mean_model = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_mean_model)

    improvement_lasso = 100 - (mse_lasso/naive_mse)*100
    improvement_lasso_boosted = 100 - (mse_lasso_boosted /naive_mse)*100
    improvement_pca_lasso = 100 - (mse_pca_lasso/naive_mse)*100
    improvement_pca_lasso_boosted = 100 - (mse_pca_lasso_boosted/naive_mse)*100
    improvement_mean_model = 100 - (mse_mean_model/naive_mse)*100
    results.loc[city,:]=[improvement_lasso, improvement_lasso_boosted, improvement_pca_lasso, improvement_pca_lasso_boosted, improvement_mean_model]
    results_mse.loc[city,:]=[naive_mse, mse_lasso, mse_lasso_boosted, mse_pca_lasso, mse_pca_lasso_boosted, mse_mean_model]
    results_r2.loc[city,:]=[naive_r2, r2_lasso, r2_lasso_boosted, r2_pca_lasso, r2_pca_lasso_boosted, r2_mean_model]
    
for city in ['hamburg', 'berlin', 'bremen']:
    country = 'DE'
    agg = get_training_data(city, country, radius, dens_type,  2015)
    
    # Lasso:
    predicts = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='predicts')
    predicts_master = predicts.copy()
    predicts_master = predicts_master.rename(columns = {'y_pred': 'y_pred_lasso'})

    # Lasso Boosted
    cols = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='used_columns')
    predicts = train_xgboost(agg, target, cols, 'predicts')
    predicts_master.loc[:, 'y_pred_lasso_boosted'] = predicts.y_pred

    # PCA Lasso
    predicts = get_best_pca_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius,output = 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso'] = predicts.y_pred

    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius, output = 'components')
    cols = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output = 'used_columns')
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(agg.iloc[:,-5:])
    predicts = train_xgboost(reduced_data, target, cols, 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso_boosted'] = predicts.y_pred

    #predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso', 'y_pred_lasso_boosted', 'y_pred_pca_lasso', 'y_pred_pca_lasso_boosted']].mean(axis = 1)
    predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted']].mean(axis = 1)

    targets = ['unemployment_rate', 'income_levels', 'foreign_nationals']
    scaler = get_training_data(city, country, radius, dens_type, 2015, 'scaler')
    scaler_new = RobustScaler()
    scaler_new.center_, scaler_new.scale_ = scaler.center_[targets.index(target)], scaler.scale_[targets.index(target)]
    predicts_master = pd.DataFrame(scaler_new.inverse_transform(predicts_master), columns =predicts_master.columns)


    naive_mse = metrics.mean_squared_error(predicts_master.y_test, predicts_master.naive) 
    mse_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso)
    mse_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    mse_pca_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    mse_pca_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    mse_mean_model = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_mean_model)
    naive_r2 = metrics.r2_score(predicts_master.y_test, predicts_master.naive) 
    
    r2_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso)
    r2_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    r2_pca_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    r2_pca_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    r2_mean_model = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_mean_model)

    improvement_lasso = 100 - (mse_lasso/naive_mse)*100
    improvement_lasso_boosted = 100 - (mse_lasso_boosted /naive_mse)*100
    improvement_pca_lasso = 100 - (mse_pca_lasso/naive_mse)*100
    improvement_pca_lasso_boosted = 100 - (mse_pca_lasso_boosted/naive_mse)*100
    improvement_mean_model = 100 - (mse_mean_model/naive_mse)*100
    results.loc[city,:]=[improvement_lasso, improvement_lasso_boosted, improvement_pca_lasso, improvement_pca_lasso_boosted, improvement_mean_model]
    results_mse.loc[city,:]=[naive_mse, mse_lasso, mse_lasso_boosted, mse_pca_lasso, mse_pca_lasso_boosted, mse_mean_model]
    results_r2.loc[city,:]=[naive_r2, r2_lasso, r2_lasso_boosted, r2_pca_lasso, r2_pca_lasso_boosted, r2_mean_model]

results.loc['mean'] = results.mean()
results

shape of training data (344, 238)
0.01
0.01
shape before pca: (344, 231)
number of pca components: 50
shape after pca: (344, 50)
0.01
shape before pca: (344, 231)
number of pca components: 50
shape after pca: (344, 50)
0.01
shape before pca: (344, 231)
number of pca components: 50
shape after pca: (344, 50)
0.01
shape of training data (344, 238)
shape of training data (176, 240)
0.1
0.1
shape before pca: (176, 233)
number of pca components: 49
shape after pca: (176, 49)
0.1
shape before pca: (176, 233)
number of pca components: 49
shape after pca: (176, 49)
0.1
shape before pca: (176, 233)
number of pca components: 49
shape after pca: (176, 49)
0.1
shape of training data (176, 240)
shape of training data (861, 241)
0.01
0.01
shape before pca: (861, 234)
number of pca components: 118
shape after pca: (861, 118)
0.01
shape before pca: (861, 234)
number of pca components: 118
shape after pca: (861, 118)
0.01
shape before pca: (861, 234)
number of pca components: 118
shape after pca: (861,

,improvement_lasso,improvement_lasso_boosted,improvement_pca_lasso,improvement_pca_lasso_boosted,improvement_mean_model
marseille,76.898929,73.435655,76.517014,72.558514,74.887293
lyon,30.236581,22.72755,34.281406,18.166401,28.264693
paris,31.169622,30.378314,33.854867,33.8764,33.980508
hamburg,13.262894,10.964235,34.04139,1.416834,14.490872
berlin,47.268591,27.308296,19.717307,46.900592,55.969617
bremen,23.152348,25.311243,41.178896,55.257949,48.466512
mean,36.998161,31.687549,39.931813,38.029448,42.676583


In [8]:
results_r2

,r2_naive,r2_lasso,r2_lasso_boosted,r2_pca_lasso,r2_pca_lasso_boosted,r2_mean_model
marseille,-0.026125,0.762954,0.727417,0.759035,0.718416,0.742312
lyon,-0.002119,0.300888,0.225638,0.341422,0.17993,0.281127
paris,-0.021716,0.296749,0.288664,0.324185,0.324405,0.325468
hamburg,-0.26129,-0.094007,-0.122999,0.168071,-0.24342,-0.078518
berlin,-0.033907,0.454806,0.248435,0.169951,0.451001,0.544767
bremen,-0.012699,0.221765,0.243628,0.40432,0.546898,0.478121


In [9]:
results_mse

,mse_naive,mse_lasso,mse_lasso_boosted,mse_pca_lasso,mse_pca_lasso_boosted,mse_mean_model
marseille,0.009139,0.002111,0.002428,0.002146,0.002508,0.002295
lyon,0.002496,0.001741,0.001929,0.00164,0.002043,0.001791
paris,0.002873,0.001978,0.002,0.0019,0.0019,0.001897
hamburg,0.005245,0.004549,0.00467,0.00346,0.005171,0.004485
berlin,0.003961,0.002089,0.00288,0.00318,0.002103,0.001744
bremen,0.006663,0.005121,0.004977,0.003919,0.002981,0.003434


In [10]:
results_rmse = results_mse**(0.5)
results_rmse = results_rmse.rename(columns = {'mse_naive': 'rmse_naive','mse_lasso':'rmse_lasso', 'mse_lasso_boosted':'rmse_lasso_boosted', 'mse_pca_lasso':'rmse_pca_lasso', 'mse_pca_lasso_boosted':'rmse_pca_lasso_boosted', 'mse_mean_model':'rmse_mean_model'})
results_rmse

,rmse_naive,rmse_lasso,rmse_lasso_boosted,rmse_pca_lasso,rmse_pca_lasso_boosted,rmse_mean_model
marseille,0.095596,0.045947,0.049271,0.046325,0.050077,0.047905
lyon,0.049962,0.041731,0.043919,0.040503,0.045197,0.042316
paris,0.053601,0.04447,0.044725,0.043594,0.043587,0.043552
hamburg,0.072423,0.067449,0.068337,0.058818,0.071908,0.06697
berlin,0.062939,0.045704,0.053662,0.056394,0.045864,0.041764
bremen,0.081628,0.071558,0.070545,0.062605,0.054601,0.058598


In [11]:
output = pd.concat([results, results_r2, results_mse, results_rmse], axis = 1, join = 'inner')
output = output[['mse_naive', 'rmse_naive', 'r2_naive',
               'mse_lasso', 'rmse_lasso', 'r2_lasso', 'improvement_lasso',
               'mse_lasso_boosted', 'rmse_lasso_boosted', 'r2_lasso_boosted', 'improvement_lasso_boosted',
               'mse_pca_lasso', 'rmse_pca_lasso', 'r2_pca_lasso', 'improvement_pca_lasso',
               'mse_pca_lasso_boosted', 'rmse_pca_lasso_boosted', 'r2_pca_lasso_boosted', 'improvement_pca_lasso_boosted',
               'mse_mean_model', 'rmse_mean_model', 'r2_mean_model', 'improvement_mean_model']]
output.to_excel(f'output/results/results_{target}_{dens_type}_{radius}.xlsx')
output

,mse_naive,rmse_naive,r2_naive,mse_lasso,rmse_lasso,r2_lasso,improvement_lasso,mse_lasso_boosted,rmse_lasso_boosted,r2_lasso_boosted,...,r2_pca_lasso,improvement_pca_lasso,mse_pca_lasso_boosted,rmse_pca_lasso_boosted,r2_pca_lasso_boosted,improvement_pca_lasso_boosted,mse_mean_model,rmse_mean_model,r2_mean_model,improvement_mean_model
marseille,0.009139,0.095596,-0.026125,0.002111,0.045947,0.762954,76.898929,0.002428,0.049271,0.727417,...,0.759035,76.517014,0.002508,0.050077,0.718416,72.558514,0.002295,0.047905,0.742312,74.887293
lyon,0.002496,0.049962,-0.002119,0.001741,0.041731,0.300888,30.236581,0.001929,0.043919,0.225638,...,0.341422,34.281406,0.002043,0.045197,0.17993,18.166401,0.001791,0.042316,0.281127,28.264693
paris,0.002873,0.053601,-0.021716,0.001978,0.04447,0.296749,31.169622,0.002,0.044725,0.288664,...,0.324185,33.854867,0.0019,0.043587,0.324405,33.8764,0.001897,0.043552,0.325468,33.980508
hamburg,0.005245,0.072423,-0.26129,0.004549,0.067449,-0.094007,13.262894,0.00467,0.068337,-0.122999,...,0.168071,34.04139,0.005171,0.071908,-0.24342,1.416834,0.004485,0.06697,-0.078518,14.490872
berlin,0.003961,0.062939,-0.033907,0.002089,0.045704,0.454806,47.268591,0.00288,0.053662,0.248435,...,0.169951,19.717307,0.002103,0.045864,0.451001,46.900592,0.001744,0.041764,0.544767,55.969617
bremen,0.006663,0.081628,-0.012699,0.005121,0.071558,0.221765,23.152348,0.004977,0.070545,0.243628,...,0.40432,41.178896,0.002981,0.054601,0.546898,55.257949,0.003434,0.058598,0.478121,48.466512


#### Income levels

In [12]:
results = pd.DataFrame(columns = ['improvement_lasso', 'improvement_lasso_boosted', 'improvement_pca_lasso', 'improvement_pca_lasso_boosted', 'improvement_mean_model'])
results_mse = pd.DataFrame(columns = ['mse_naive','mse_lasso', 'mse_lasso_boosted', 'mse_pca_lasso', 'mse_pca_lasso_boosted', 'mse_mean_model'])
results_r2 = pd.DataFrame(columns = ['r2_naive', 'r2_lasso', 'r2_lasso_boosted', 'r2_pca_lasso', 'r2_pca_lasso_boosted', 'r2_mean_model'])
target = 'income_levels'
dens_type = 'score'
radius = 1000

for city in ['marseille', 'lyon', 'paris']:
    country = 'FR'
    agg = get_training_data(city, country, radius, dens_type,  2015)
    
    # Lasso:
    predicts = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='predicts')
    predicts_master = predicts.copy()
    predicts_master = predicts_master.rename(columns = {'y_pred': 'y_pred_lasso'})

    # Lasso Boosted
    cols = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='used_columns')
    predicts = train_xgboost(agg, target, cols, 'predicts')
    predicts_master.loc[:, 'y_pred_lasso_boosted'] = predicts.y_pred

    # PCA Lasso
    predicts = get_best_pca_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius,output = 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso'] = predicts.y_pred

    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius, output = 'components')
    cols = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius, output = 'used_columns')
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(agg.iloc[:,-5:])
    predicts = train_xgboost(reduced_data, target, cols, 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso_boosted'] = predicts.y_pred

    #predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso', 'y_pred_lasso_boosted', 'y_pred_pca_lasso', 'y_pred_pca_lasso_boosted']].mean(axis = 1)
    predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted']].mean(axis = 1)


    targets = ['unemployment_rate', 'income_levels', 'foreign_nationals']
    scaler = get_training_data(city, country, radius, dens_type, 2015, 'scaler')
    scaler_new = RobustScaler()
    scaler_new.center_, scaler_new.scale_ = scaler.center_[targets.index(target)], scaler.scale_[targets.index(target)]
    predicts_master = pd.DataFrame(scaler_new.inverse_transform(predicts_master), columns =predicts_master.columns)


    naive_mse = metrics.mean_squared_error(predicts_master.y_test, predicts_master.naive) 
    mse_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso)
    mse_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    mse_pca_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    mse_pca_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    mse_mean_model = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_mean_model)
    
    naive_r2 = metrics.r2_score(predicts_master.y_test, predicts_master.naive) 
    r2_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso)
    r2_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    r2_pca_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    r2_pca_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    r2_mean_model = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_mean_model)

    improvement_lasso = 100 - (mse_lasso/naive_mse)*100
    improvement_lasso_boosted = 100 - (mse_lasso_boosted /naive_mse)*100
    improvement_pca_lasso = 100 - (mse_pca_lasso/naive_mse)*100
    improvement_pca_lasso_boosted = 100 - (mse_pca_lasso_boosted/naive_mse)*100
    improvement_mean_model = 100 - (mse_mean_model/naive_mse)*100
    results.loc[city,:]=[improvement_lasso, improvement_lasso_boosted, improvement_pca_lasso, improvement_pca_lasso_boosted, improvement_mean_model]
    results_mse.loc[city,:]=[naive_mse, mse_lasso, mse_lasso_boosted, mse_pca_lasso, mse_pca_lasso_boosted, mse_mean_model]
    results_r2.loc[city,:]=[naive_r2, r2_lasso, r2_lasso_boosted, r2_pca_lasso, r2_pca_lasso_boosted, r2_mean_model]
    
for city in ['hamburg', 'berlin', 'bremen']:
    country = 'DE'
    agg = get_training_data(city, country, radius, dens_type,  2015)
    
    # Lasso:
    predicts = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='predicts')
    predicts_master = predicts.copy()
    predicts_master = predicts_master.rename(columns = {'y_pred': 'y_pred_lasso'})

    # Lasso Boosted
    cols = get_best_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output ='used_columns')
    predicts = train_xgboost(agg, target, cols, 'predicts')
    predicts_master.loc[:, 'y_pred_lasso_boosted'] = predicts.y_pred

    # PCA Lasso
    predicts = get_best_pca_lasso_model(agg=agg, target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius,output = 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso'] = predicts.y_pred

    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015,  density_type=dens_type, radius = radius, output = 'components')
    cols = get_best_pca_lasso_model(agg=agg,target=target, city=city, country=country, socio_year=2015, density_type=dens_type, radius = radius, output = 'used_columns')
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(agg.iloc[:,-5:])
    predicts = train_xgboost(reduced_data, target, cols, 'predicts')
    predicts_master.loc[:,'y_pred_pca_lasso_boosted'] = predicts.y_pred

    #predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso', 'y_pred_lasso_boosted', 'y_pred_pca_lasso', 'y_pred_pca_lasso_boosted']].mean(axis = 1)
    predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted']].mean(axis = 1)

    targets = ['unemployment_rate', 'income_levels', 'foreign_nationals']
    scaler = get_training_data(city, country, radius, dens_type, 2015, 'scaler')
    scaler_new = RobustScaler()
    scaler_new.center_, scaler_new.scale_ = scaler.center_[targets.index(target)], scaler.scale_[targets.index(target)]
    predicts_master = pd.DataFrame(scaler_new.inverse_transform(predicts_master), columns =predicts_master.columns)


    naive_mse = metrics.mean_squared_error(predicts_master.y_test, predicts_master.naive) 
    mse_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso)
    mse_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    mse_pca_lasso = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    mse_pca_lasso_boosted = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    mse_mean_model = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_mean_model)
    
    naive_r2 = metrics.r2_score(predicts_master.y_test, predicts_master.naive) 
    r2_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso)
    r2_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_lasso_boosted)
    r2_pca_lasso = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso)
    r2_pca_lasso_boosted = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_pca_lasso_boosted)
    r2_mean_model = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_mean_model)

    improvement_lasso = 100 - (mse_lasso/naive_mse)*100
    improvement_lasso_boosted = 100 - (mse_lasso_boosted /naive_mse)*100
    improvement_pca_lasso = 100 - (mse_pca_lasso/naive_mse)*100
    improvement_pca_lasso_boosted = 100 - (mse_pca_lasso_boosted/naive_mse)*100
    improvement_mean_model = 100 - (mse_mean_model/naive_mse)*100
    results.loc[city,:]=[improvement_lasso, improvement_lasso_boosted, improvement_pca_lasso, improvement_pca_lasso_boosted, improvement_mean_model]
    results_mse.loc[city,:]=[naive_mse, mse_lasso, mse_lasso_boosted, mse_pca_lasso, mse_pca_lasso_boosted, mse_mean_model]
    results_r2.loc[city,:]=[naive_r2, r2_lasso, r2_lasso_boosted, r2_pca_lasso, r2_pca_lasso_boosted, r2_mean_model]

results.loc['mean'] = results.mean()
results

shape of training data (344, 238)
0.01
0.01
shape before pca: (344, 231)
number of pca components: 50
shape after pca: (344, 50)
0.01
shape before pca: (344, 231)
number of pca components: 50
shape after pca: (344, 50)
0.01
shape before pca: (344, 231)
number of pca components: 50
shape after pca: (344, 50)
0.01
shape of training data (344, 238)
shape of training data (176, 240)
0.1
0.1
shape before pca: (176, 233)
number of pca components: 49
shape after pca: (176, 49)
0.1
shape before pca: (176, 233)
number of pca components: 49
shape after pca: (176, 49)
0.1
shape before pca: (176, 233)
number of pca components: 49
shape after pca: (176, 49)
0.1
shape of training data (176, 240)
shape of training data (861, 241)
0.001
0.001
shape before pca: (861, 234)
number of pca components: 118
shape after pca: (861, 118)
0.01
shape before pca: (861, 234)
number of pca components: 118
shape after pca: (861, 118)
0.01
shape before pca: (861, 234)
number of pca components: 118
shape after pca: (86

,improvement_lasso,improvement_lasso_boosted,improvement_pca_lasso,improvement_pca_lasso_boosted,improvement_mean_model
marseille,77.972113,79.269035,73.932664,72.114245,78.967522
lyon,56.221045,70.423961,58.616177,48.302743,62.895756
paris,74.685205,77.448456,73.455905,70.167286,76.36102
hamburg,32.502566,49.666675,17.001813,7.597564,36.076701
berlin,11.566901,37.957415,16.974063,6.114116,32.968867
bremen,38.617818,44.395172,33.50568,40.809613,46.356178
mean,48.594275,59.860119,45.58105,40.850928,55.604341


In [13]:
results_r2

,r2_naive,r2_lasso,r2_lasso_boosted,r2_pca_lasso,r2_pca_lasso_boosted,r2_mean_model
marseille,-0.000792,0.779547,0.792526,0.73912,0.720921,0.789509
lyon,-0.000858,0.561835,0.703986,0.585807,0.482584,0.628639
paris,-0.005564,0.745444,0.77323,0.733082,0.700013,0.762295
hamburg,-0.042585,0.296282,0.475233,0.134674,0.036626,0.333546
berlin,-0.001062,0.114729,0.378915,0.168859,0.060144,0.328976
bremen,-0.205356,0.260126,0.329764,0.198507,0.286545,0.353401


In [14]:
results_mse

,mse_naive,mse_lasso,mse_lasso_boosted,mse_pca_lasso,mse_pca_lasso_boosted,mse_mean_model
marseille,361248.317935,79575.369628,74890.261937,94167.812566,100736.821957,75979.47159
lyon,244401.878031,106996.587287,72284.395599,101142.841676,126349.067629,90683.468414
paris,840140.3386,212679.802865,189464.61614,223007.647507,250636.660461,198600.60272
hamburg,667237.806043,450368.398383,335842.972794,553795.282666,616543.987144,426520.414955
berlin,731046.943231,646487.466381,453560.422263,606958.574046,686349.886169,490029.048142
bremen,626827.178684,384760.202495,348546.176363,416804.46866,371021.432266,336254.054908


In [15]:
results_rmse = results_mse**(0.5)
results_rmse = results_rmse.rename(columns = {'mse_naive': 'rmse_naive','mse_lasso':'rmse_lasso', 'mse_lasso_boosted':'rmse_lasso_boosted', 'mse_pca_lasso':'rmse_pca_lasso', 'mse_pca_lasso_boosted':'rmse_pca_lasso_boosted', 'mse_mean_model':'rmse_mean_model'})
results_rmse

,rmse_naive,rmse_lasso,rmse_lasso_boosted,rmse_pca_lasso,rmse_pca_lasso_boosted,rmse_mean_model
marseille,601.039365,282.091066,273.660852,306.867744,317.390646,275.64374
lyon,494.370183,327.103328,268.857575,318.029624,355.45614,301.13696
paris,916.591697,461.172205,435.275334,472.236855,500.636256,445.646275
hamburg,816.846256,671.094925,579.519605,744.174229,785.20315,653.085305
berlin,855.012832,804.044443,673.468947,779.075461,828.462363,700.020748
bremen,791.724181,620.290418,590.377994,645.603957,609.115287,579.874172


In [16]:
output = pd.concat([results, results_r2, results_mse, results_rmse], axis = 1, join = 'inner')
output = output[['mse_naive', 'rmse_naive', 'r2_naive',
               'mse_lasso', 'rmse_lasso', 'r2_lasso', 'improvement_lasso',
               'mse_lasso_boosted', 'rmse_lasso_boosted', 'r2_lasso_boosted', 'improvement_lasso_boosted',
               'mse_pca_lasso', 'rmse_pca_lasso', 'r2_pca_lasso', 'improvement_pca_lasso',
               'mse_pca_lasso_boosted', 'rmse_pca_lasso_boosted', 'r2_pca_lasso_boosted', 'improvement_pca_lasso_boosted',
               'mse_mean_model', 'rmse_mean_model', 'r2_mean_model', 'improvement_mean_model']]
output.to_excel(f'output/results/results_{target}_{dens_type}_{radius}.xlsx')
output

,mse_naive,rmse_naive,r2_naive,mse_lasso,rmse_lasso,r2_lasso,improvement_lasso,mse_lasso_boosted,rmse_lasso_boosted,r2_lasso_boosted,...,r2_pca_lasso,improvement_pca_lasso,mse_pca_lasso_boosted,rmse_pca_lasso_boosted,r2_pca_lasso_boosted,improvement_pca_lasso_boosted,mse_mean_model,rmse_mean_model,r2_mean_model,improvement_mean_model
marseille,361248.317935,601.039365,-0.000792,79575.369628,282.091066,0.779547,77.972113,74890.261937,273.660852,0.792526,...,0.73912,73.932664,100736.821957,317.390646,0.720921,72.114245,75979.47159,275.64374,0.789509,78.967522
lyon,244401.878031,494.370183,-0.000858,106996.587287,327.103328,0.561835,56.221045,72284.395599,268.857575,0.703986,...,0.585807,58.616177,126349.067629,355.45614,0.482584,48.302743,90683.468414,301.13696,0.628639,62.895756
paris,840140.3386,916.591697,-0.005564,212679.802865,461.172205,0.745444,74.685205,189464.61614,435.275334,0.77323,...,0.733082,73.455905,250636.660461,500.636256,0.700013,70.167286,198600.60272,445.646275,0.762295,76.36102
hamburg,667237.806043,816.846256,-0.042585,450368.398383,671.094925,0.296282,32.502566,335842.972794,579.519605,0.475233,...,0.134674,17.001813,616543.987144,785.20315,0.036626,7.597564,426520.414955,653.085305,0.333546,36.076701
berlin,731046.943231,855.012832,-0.001062,646487.466381,804.044443,0.114729,11.566901,453560.422263,673.468947,0.378915,...,0.168859,16.974063,686349.886169,828.462363,0.060144,6.114116,490029.048142,700.020748,0.328976,32.968867
bremen,626827.178684,791.724181,-0.205356,384760.202495,620.290418,0.260126,38.617818,348546.176363,590.377994,0.329764,...,0.198507,33.50568,371021.432266,609.115287,0.286545,40.809613,336254.054908,579.874172,0.353401,46.356178
